In [6]:
# Generate inital dataframe
import pandas as pd
from faker import Faker
import random

fake = Faker()
num_rows = 10
ids = [fake.uuid4() for _ in range(num_rows)]
dates = [fake.date_between(start_date='-1y', end_date='-1m').strftime('%Y-%m-%d') for _ in range(num_rows)]
metric_values = [round(random.uniform(0, 10),2) for _ in range(num_rows)]
dataframe = pd.DataFrame({'unique_key': ids, 'date': dates, 'metric_value': metric_values})

# Insert duplication
n = 2  # Number of rows to duplicate
indices = random.sample(range(len(dataframe)), n)
subset = dataframe.iloc[indices]

# Append the selected rows to the DataFrame
dataframe = pd.concat([dataframe, subset])
len(dataframe)

12

In [7]:

def rename_duplicates(df):
    """
    Rename duplicate 'unique_key' values in the DataFrame.

    Parameters:
    - df: pandas DataFrame with a 'unique_key' column.

    Returns:
    - DataFrame with renamed duplicates.
    """
    
    # Find duplicated rows based on the 'unique_key' column
    duplicates = df['unique_key'].duplicated(keep="first")
    
    # Create a series with the same index as the dataframe for counting duplicates
    counter = df[duplicates].groupby('unique_key').cumcount() + 1
    
    # Format the 'unique_key' for duplicates
    df.loc[duplicates, 'unique_key'] = df['unique_key'][duplicates] + "-duplicate-" + counter.astype(str)

    return df

renamed_dataframe = rename_duplicates(dataframe)
print(renamed_dataframe)

                                         unique_key        date  metric_value
0              352a4c99-4927-4737-8f84-bdee0ebf168f  2023-08-12          8.81
1              c3f65317-c4cc-41b9-a442-d61714b8d5b3  2022-12-24          3.74
2              5a63728c-cd2b-45c6-8b3e-da73e3756fcb  2023-10-18          7.22
3              0a49f6b0-33eb-4ef4-a9f9-f5259b5f4cbc  2023-03-04          5.04
4              c1337b92-113d-42bb-8f94-95b36d1b502f  2023-03-20          4.26
5              ab540531-d19e-4c6c-aa29-cb0a543e006b  2023-10-14          2.22
6              fc088dff-d2af-4c4f-84ed-c9f0987fbf7c  2023-05-03          1.88
7              5b08d407-714c-4cfa-8686-e7bd2296e42f  2023-09-07          1.95
8              b351a061-3ac6-426a-b120-69d07346a6ed  2023-03-14          0.45
9              a7c4c075-7d41-45e7-9a4d-396375ae3f79  2023-05-08          7.14
7  5b08d407-714c-4cfa-8686-e7bd2296e42f-duplicate-1  2023-09-07          1.95
6  fc088dff-d2af-4c4f-84ed-c9f0987fbf7c-duplicate-1  2023-05-03 

In [8]:
from dotenv import load_dotenv
from github import Github
import os
import sys
load_dotenv("../../.env")
gh_token = os.getenv("GH_TOKEN")
gh_client = Github(gh_token, timeout=60)
repo = os.getenv("REPO") or "gh_org/repo"
file_path = repo+"/duplicates/test.csv"



In [9]:

from driftdb.connectors import GithubConnector

github_connector = GithubConnector(github_client=gh_client, github_repository_name=repo, default_branch="test")

github_connector.snapshot_table( table_dataframe=dataframe, table_name=file_path, )

driftdb.connectors.github_connector - INFO - Table not found. Creating it
driftdb.connectors.github_connector - INFO - Creating table on branch: test
driftdb.connectors.github_connector - INFO - Commit: New data: Samox/data-history/duplicates/test.csv
driftdb.connectors.github_connector - INFO - Table stored
